In [1]:
import pandas as pd
import re

In [2]:
# --- Day 5: Binary Boarding ---
# You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

# You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

# Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

# The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

# For example, consider just the first seven characters of FBFBBFFRLR:

# Start by considering the whole range, rows 0 through 127.
# F means to take the lower half, keeping rows 0 through 63.
# B means to take the upper half, keeping rows 32 through 63.
# F means to take the lower half, keeping rows 32 through 47.
# B means to take the upper half, keeping rows 40 through 47.
# B keeps rows 44 through 47.
# F keeps rows 44 through 45.
# The final F keeps the lower of the two, row 44.
# The last three characters will be either L or R; 
# these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). 
# The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

# For example, consider just the last 3 characters of FBFBBFFRLR:

# Start by considering the whole range, columns 0 through 7.
# R means to take the upper half, keeping columns 4 through 7.
# L means to take the lower half, keeping columns 4 through 5.
# The final R keeps the upper of the two, column 5.
# So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

# Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

# Here are some other boarding passes:

# BFFFBBFRRR: row 70, column 7, seat ID 567.
# FFFBBBFRRR: row 14, column 7, seat ID 119.
# BBFFBBFRLL: row 102, column 4, seat ID 820.
# As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

In [3]:
df = pd.read_excel('seat_numbers.xlsx')
df.head()

,seat_number_code
0,BBFFFFBRLL
1,BFBBBBFLLL
2,FBBBFBFLLR
3,BFBBBFBLRR
4,FBBFFBFLRR


In [4]:
df['row_number_code'] = df.seat_number_code.str[0:7]
df['column_number_code'] = df.seat_number_code.str[-3:]

In [5]:
df.head()

,seat_number_code,row_number_code,column_number_code
0,BBFFFFBRLL,BBFFFFB,RLL
1,BFBBBBFLLL,BFBBBBF,LLL
2,FBBBFBFLLR,FBBBFBF,LLR
3,BFBBBFBLRR,BFBBBFB,LRR
4,FBBFFBFLRR,FBBFFBF,LRR


In [6]:
# convert to (U)pper and (L)ower

def upper_lower(value):

    value = re.sub(r'B', 'U', value)
    value = re.sub(r'F', 'L', value)
    value = re.sub(r'R', 'U', value)

    return value

In [7]:
# F means to take the lower half, keeping rows 0 through 63.
# B means to take the upper half, keeping rows 32 through 63.

df['row_number_code'] = df.row_number_code.apply(upper_lower)

# R means to take the upper half, keeping columns 4 through 7.
# L means to take the lower half, keeping columns 4 through 5.
    
df['column_number_code'] = df.column_number_code.apply(upper_lower)

In [8]:
df.head()

,seat_number_code,row_number_code,column_number_code
0,BBFFFFBRLL,UULLLLU,ULL
1,BFBBBBFLLL,ULUUUUL,LLL
2,FBBBFBFLLR,LUUULUL,LLU
3,BFBBBFBLRR,ULUUULU,LUU
4,FBBFFBFLRR,LUULLUL,LUU


In [9]:
def get_number_row(code):
    
    rows = list(range(0,128))

    n = 64

    for letter in code:

        if letter == 'U': 

            for r in range(0,n):
                rows.remove(min(rows))

        if letter == 'L': 

            for r in range(0,n):
                rows.remove(max(rows))

        n //=2
        
    return rows[0]

In [10]:
def get_number_column(code):
    
    columns = list(range(0,8))

    n = 4

    for letter in code:

        if letter == 'U': 

            for r in range(0,n):
                columns.remove(min(columns))

        if letter == 'L': 

            for r in range(0,n):
                columns.remove(max(columns))

        n //=2 
        
    return columns[0]

In [11]:
df['row_number'] = df.row_number_code.apply(get_number_row)
df['column_number'] = df.column_number_code.apply(get_number_column)

In [12]:
df.head()

,seat_number_code,row_number_code,column_number_code,row_number,column_number
0,BBFFFFBRLL,UULLLLU,ULL,97,4
1,BFBBBBFLLL,ULUUUUL,LLL,94,0
2,FBBBFBFLLR,LUUULUL,LLU,58,1
3,BFBBBFBLRR,ULUUULU,LUU,93,3
4,FBBFFBFLRR,LUULLUL,LUU,50,3


In [13]:
df['seat_number'] = (df.row_number * 8) + df.column_number

In [14]:
df.head()

,seat_number_code,row_number_code,column_number_code,row_number,column_number,seat_number
0,BBFFFFBRLL,UULLLLU,ULL,97,4,780
1,BFBBBBFLLL,ULUUUUL,LLL,94,0,752
2,FBBBFBFLLR,LUUULUL,LLU,58,1,465
3,BFBBBFBLRR,ULUUULU,LUU,93,3,747
4,FBBFFBFLRR,LUULLUL,LUU,50,3,403


In [15]:
df.seat_number.max()

883

In [16]:
df.seat_number.min()

85

In [17]:
# --- Part Two ---
# Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

# It's a completely full flight, so your seat should be the only missing boarding pass in your list. 
# However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

# Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

# What is the ID of your seat?

In [18]:
seat_ids = list(df.seat_number)
seat_ids

[780,
 752,
 465,
 747,
 403,
 151,
 376,
 581,
 580,
 726,
 267,
 809,
 845,
 677,
 302,
 533,
 253,
 553,
 508,
 430,
 338,
 180,
 662,
 489,
 211,
 694,
 281,
 407,
 90,
 494,
 690,
 810,
 770,
 869,
 190,
 261,
 99,
 546,
 653,
 851,
 487,
 276,
 258,
 523,
 114,
 372,
 187,
 260,
 131,
 862,
 91,
 607,
 202,
 742,
 196,
 299,
 444,
 397,
 290,
 95,
 796,
 703,
 803,
 152,
 112,
 446,
 849,
 644,
 613,
 709,
 798,
 659,
 399,
 137,
 541,
 823,
 744,
 235,
 146,
 125,
 655,
 806,
 611,
 88,
 496,
 657,
 863,
 365,
 404,
 714,
 710,
 309,
 442,
 289,
 109,
 409,
 118,
 870,
 229,
 543,
 501,
 847,
 596,
 642,
 463,
 432,
 529,
 777,
 163,
 478,
 562,
 272,
 773,
 116,
 236,
 589,
 819,
 491,
 586,
 133,
 510,
 393,
 257,
 436,
 582,
 401,
 201,
 477,
 625,
 215,
 92,
 286,
 570,
 824,
 362,
 794,
 490,
 816,
 359,
 722,
 306,
 682,
 730,
 105,
 86,
 471,
 457,
 563,
 547,
 791,
 107,
 518,
 385,
 839,
 455,
 645,
 708,
 233,
 624,
 278,
 135,
 246,
 139,
 495,
 800,
 481,
 304,
 748,

In [19]:
df.column_number.value_counts()

7    100
6    100
5    100
3    100
2    100
1    100
0    100
4     98
Name: column_number, dtype: int64

In [20]:
df.row_number.value_counts()

60     8
36     8
38     8
39     8
40     8
      ..
83     8
12     8
66     7
110    4
10     3
Name: row_number, Length: 101, dtype: int64

In [21]:
df_4 = df[df.column_number == 4]
df_4

,seat_number_code,row_number_code,column_number_code,row_number,column_number,seat_number
0,BBFFFFBRLL,UULLLLU,ULL,97,4,780
8,BFFBFFFRLL,ULLULLL,ULL,72,4,580
18,FBBBBBBRLL,LUUUUUU,ULL,63,4,508
21,FFBFBBFRLL,LLULUUL,ULL,22,4,180
41,FBFFFBFRLL,LULLLUL,ULL,34,4,276
...,...,...,...,...,...,...
756,BBFBBFFRLL,UULUULL,ULL,108,4,868
757,FFFBBFFRLL,LLLUULL,ULL,12,4,100
780,BFFBBBBRLL,ULLUUUU,ULL,79,4,636
785,BFFFBBBRLL,ULLLUUU,ULL,71,4,572


In [22]:
df.row_number.value_counts()

60     8
36     8
38     8
39     8
40     8
      ..
83     8
12     8
66     7
110    4
10     3
Name: row_number, Length: 101, dtype: int64

In [23]:
li = []
rows = [66,110,10]
column =4

for number in rows:
    
    li.append((number*8)+column)
li

[532, 884, 84]

In [27]:
li = [532, 884, 84]

seat_ids = list(df.seat_number)

for id_num in li:
    
    id_up = id_num + 1
    
    id_down = id_num - 1
        
    if (id_up in seat_ids) & (id_down in seat_ids):
        
        print(id_num)

532
